# Goal : 輸入一個句子，判斷是"正面" or "負面" 

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn.functional as f

/home/uscc/.pyenv/versions/3.6.8/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def load_train_data(path='training_label.txt'):
    # 把 training 時需要的 data 讀進來
    # 如果是 'training_label.txt'，需要讀取 label，如果是 'training_nolabel.txt'，不需要讀取 label
    if 'training_label.txt' in path:
        with open(path,'r') as f:
            lines = f.readlines()
            lines = [line.strip('\n').split(' ') for line in lines]
        x = [line[2:] for line in lines]
        y = [line[0] for line in lines]
        
        return x,y
    else:
        with open(path,'r') as f:
            lines = f.readlines()
            x = [line.strip('\n').split(' ') for line in lines]
        
        return x

def load_test_date(path='testing_data.txt'):
    with open(path,'r') as f:
        f.readline()
        lines = f.readlines()
        # join 是要把因為逗號切開的斷句合起來
        X = ["".join(line.strip('\n').split(',')[1:]).strip() for line in lines]
        X = [sen.split(' ') for sen in X]
    return X

def evaluation(outputs, labels):
    # outputs => probability (float)
    # labels => labels
    outputs[outputs >= 0.5] = 1
    outputs[outputs < 0.5] = 0
    # item方法的使用:
    # 如果tensor只有一個元素那麼呼叫item方法的時候就是將tensor轉換成python的scalars;
    # 如果tensor不是單個元素的話那就會引發ValueError
    correct = torch.sum(torch.eq(outputs,labels)).item()
    return correct

## [Word2Vec 說明文章 ](https://medium.com/@tengyuanchang/讓電腦聽懂人話-理解-nlp-重要技術-word2vec-的-skip-gram-模型-73d0239ad698)
## Train word to vector

In [3]:
from gensim.models import word2vec


def train_word2vec(x):
    # sg = 1 : 使用skip-gram (拿wi 預測 wi-1 與 wi+1)
    # size : word embeddingvertor 的維度，Dimensionality of the word vectors.
    # window : Maximum distance between the current and predicted word within a sentence.
    # min_count : Ignores all words with total frequency lower than set_num.
    # workers : Use these many worker threads to train the model (=faster training with multicore machines).
    # iter : Number of iterations (epochs) over the corpus(語意庫).
    model = word2vec.Word2Vec(x, size=250, window=5, min_count=5, workers=12, iter=10, sg=1)
    return model

In [4]:
print('loading training data...')
train_x ,train_y = load_train_data()
train_x_nolable = load_train_data('training_nolabel.txt')

print('loading testing data...')
test_x = load_test_date()

# 使用train_x 與 test_x 來得到word embedding 
model = train_word2vec(train_x + test_x)

print('saving model...')
model.save('./w2v_all.model')

loading training data...
loading testing data...
saving model...


## Data preprocess 

In [5]:
from torch import nn

class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path='./w2v.model'):
        self.w2v_path=w2v_path
        self.sentences=sentences
        self.sen_len=sen_len
        self.idx2word=[]
        self.word2idx={}
        self.embedding_matrix=[]
    
    def get_w2v_model(self):
        self.embedding = word2vec.Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
    
    def add_embedding(self, word):
        # 把 word 加進 embedding，並賦予他一個隨機生成的 representation vector
        # word 只會是 "<PAD>" 或 "<UNK>"
        vector = torch.empty(1, self.embedding_dim)
        nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix,vector] , 0)
    
    def make_embedding(self, load=True):
        print('Get embedding...')
        # 取得訓練好的 Word2vec word embedding
        if load:
            print('loading word to vec model...')
            self.get_w2v_model()
        
        else:
            raise NotImplementedError
        # 製作一個 word2idx 的 dictionary
        # 製作一個 idx2word 的 list
        # 製作一個 word2vector 的 list
        # wv.vocab word vector的值
        for i, word in enumerate(self.embedding.wv.vocab):
            print('get words#{}'.format(i+1), end='\r')
            #e.g. self.word2index['he'] = 1 
            #e.g. self.index2word[1] = 'he'
            #e.g. self.vectors[1] = 'he' vector
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
        
        print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        # 將 "<PAD>" 跟 "<UNK>" 加進 embedding 裡面
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words : {}".format(len(self.embedding_matrix)))
        
        return self.embedding_matrix

    def pad_sequence(self, sentence):
        # 將每個句子變長一樣長度
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        
        assert len(sentence) == self.sen_len
        return sentence
    
    def sentence_word2idx(self):
        # 把句子裡的字轉成對應的 index
        sentence_list = []
        for i , sen in enumerate(self.sentences):
            print('sentence count #{}'.format(i+1), end='\r')
            sentence_index = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_index.append(self.word2idx[word])
                else:
                    sentence_index.append(self.word2idx["<UNK>"])
            #將句子變成一樣長度
            sentence_index = self.pad_sequence(sentence_index)
            sentence_list.append(sentence_index)
        return torch.LongTensor(sentence_list)
    
    def labels_to_tensor(self, y):
        #把label 轉成 tensor
        y = [int(label) for label in y]
        return torch.LongTensor(y)
                


## Dataset 

In [6]:
from torch.utils.data import Dataset

class TwitterDataset(Dataset):
    def __init__(self, x, y):
        self.data = x
        self.label = y
    
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx]
        return self.data[idx], self.label[idx]
    
    def __len__(self):
        return len(self.data)
        

## Model
- # 將index 取得對應的vector 請看[這篇筆記中的 Sharing Patameters](https://hackmd.io/cdjKzwLvRCOTrdvX5yZGlw#ML-Lecture-10--Unsupervised-Learning-Word-Embedding)

In [7]:
# 拿來訓練的模型
class LSTM_net(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super(LSTM_net, self).__init__()
        # 製作embedding layer
        self.embedding = nn.Embedding(embedding.size(0), embedding.size(1))
        self.embedding.weight = nn.Parameter(embedding)
        # 是否將 embedding fix 住，如果fix_embedding 為False ，在訓練過程中，embedding 也會跟著被改變
        self.embedding.weight.requires_grad = False if fix_embedding else True
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.classifier = nn.Sequential(nn.Dropout(dropout),
                                        nn.Linear(hidden_dim, 1),
                                        nn.Sigmoid())
        
    def forward(self, inputs):
        # 將index 取得對應的vector  
        inputs = self.embedding(inputs)
        x, _ = self.lstm(inputs, None)
        # x 的 dimension (batch, seq_len, hidden_size)
        # 取用 LSTM 最後一層的 hidden state
        x = x[:, -1, :]
        x = self.classifier(x)
        
        return x
        
        
        

## Train 

In [16]:
def training(batch_size, n_epoch, lr, model_dir, train, valid, model, device):
    # numel() : return elements num
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(total, trainable))
    
    model.train() #將model 設為train模式，這樣optimizer 可以更新 model 參數
    criterion = nn.BCELoss() #定義損失函數，這裡我們用binary cross entropy
    t_batch = len(train)
    v_batch = len(valid)
    optimizer = optim.Adam(model.parameters(), lr=lr) #將模型的參數給optimizer ，給予適當的lr
    total_loss, total_acc, best_acc = 0, 0, 0
    for epoch in range(n_epoch):
        total_loss, total_acc = 0, 0
        #這段做training
        for i, (inputs, labels) in enumerate(train):
            # device 為 "cuda"，將 inputs 轉成 torch.cuda.LongTensor
            inputs = inputs.to(device, dtype=torch.long) 
            # device為 "cuda"，將 labels 轉成 torch.cuda.FloatTensor，因為等等要餵進 criterion，所以型態要是 float
            labels = labels.to(device, dtype=torch.float) 
            
            # 由於 loss.backward() 的 gradient 會累加，所以每次餵完一個 batch 後需要歸零
            optimizer.zero_grad()
            outputs = model(inputs) # 將 input 餵給模型
            outputs = outputs.squeeze() # 去掉最外面的 dimension，好讓 outputs 可以餵進 criterion()
            loss = criterion(outputs, labels) # 計算此時模型的 training loss
            loss.backward() # 算loss 的 gradient
            optimizer.step() # 更新模型的參數
            
            correct = evaluation(outputs, labels) # 計算此時模型的 training accuracy
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            
            print('[ Epoch{}:{}/{} ] loss:{:.3f} acc:{:.3f} '.format(
                epoch+1, i+1, t_batch, loss.item(), correct/batch_size), end = '\r')
        print('\nTrain | Loss:{:.5f} Acc:{:.3f}'.format(total_loss/t_batch, total_acc/t_batch))
        
    
        #這段做validation
        model.eval() # 將 model 的模式設為 eval，這樣 model 的參數就會固定住
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i , (inputs, labels) in enumerate(valid):
                # device 為 "cuda"，將 inputs 轉成 torch.cuda.LongTensor
                inputs = inputs.to(device, dtype=torch.long) 
                # device為 "cuda"，將 labels 轉成 torch.cuda.FloatTensor，因為等等要餵進 criterion，所以型態要是 float
                labels = labels.to(device, dtype=torch.float) 
                
                outputs = model(inputs)# 將 input 餵給模型
                outputs = outputs.squeeze()# 去掉最外面的 dimension，好讓 outputs 可以餵進 criterion()
                loss = criterion(outputs, labels)
                correct = evaluation(outputs, labels)
                total_acc += (correct/batch_size)
                total_loss += loss.item()
                
            print('Valid | Loss:{:.5f} Acc:{:.3f}'.format(total_loss/v_batch, total_acc/v_batch))
            
            # 如果 validation 的結果優於之前所有的結果，就把當下的模型存下來以備之後做預測時使用
            if total_acc > best_acc:
                best_acc = total_acc
                torch.save(model, "./ckpt.model")
                print('saving model with acc:{:.3f}'.format(best_acc/v_batch))
        
        print('-'*60)
        model.train() # 將 model 的模式設為 train，這樣 optimizer 就可以更新 model 的參數（因為剛剛轉成 eval 模式）

## Test 

In [19]:
def testing(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            outputs[outputs >= 0.5] = 1
            outputs[outputs < 0.5] = 0
            ret_output += outputs.int().tolist()
        
    return ret_output
    

## Main 

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義句子長度、要不要固定 embedding、batch 大小、要訓練幾個 epoch、learning rate 的值、model 的資料夾路徑
sen_len = 20
fix_embeddding = True # fix embedding during training
batch_size = 128
epoch = 5
lr = 0.001

print('loading data...')
train_x, train_y = load_train_data()
train_x_nolable = load_train_data('training_nolabel.txt')

# 對input 跟 label 做預處理
preprocess = Preprocess(train_x, sen_len, './w2v_all.model')
embedding = preprocess.make_embedding(load = True)
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(train_y)

# 製作一個model的對象
model = LSTM_net(embedding, embedding_dim=250, hidden_dim=150, num_layers=1, dropout=0.5, fix_embedding=fix_embeddding)
model = model.to(device)

# 把data分為 train , val
x_train , x_val, y_train, y_val = train_x[:180000], train_x[180000:], y[:180000], y[180000:]

# 把data做成dataset 給 dataloader用
train_dataset = TwitterDataset(x = x_train, y = y_train)
val_dataset = TwitterDataset(x = x_val, y = y_val)

# 把data轉成 batch of tensor
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=8)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=8)

#開始訓練
training(batch_size, epoch, lr, '.', train_loader, val_loader, model, device)

loading data...
Get embedding...
loading word to vec model...


/home/uscc/.pyenv/versions/3.6.8/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


get words#3533

get words#4372

get words#5248

...

get words#20822

get words#21702

get words#22562

get words#24333

get words#24694
total words : 24696
sentence count #200000
start training, parameter total:6415351, trainable:241351

[ Epoch1:1407/1407 ] loss:0.609 acc:0.188 
Train | Loss:0.49582 Acc:0.753
Valid | Loss:0.45338 Acc:0.782
saving model with acc:0.782
------------------------------------------------------------
[ Epoch2:1407/1407 ] loss:0.440 acc:0.195 
Train | Loss:0.44420 Acc:0.790
Valid | Loss:0.44168 Acc:0.792
saving model with acc:0.792
------------------------------------------------------------
[ Epoch3:1407/1407 ] loss:0.403 acc:0.203 
Train | Loss:0.42753 Acc:0.800
Valid | Loss:0.43644 Acc:0.794
saving model with acc:0.794
------------------------------------------------------------
[ Epoch4:1407/1407 ] loss:0.485 acc:0.195 
Train | Loss:0.41454 Acc:0.808
Valid | Loss:0.42443 Acc:0.800
saving model with acc:0.800
------------------------------------------------------------
[ Epoch5:1407/1407 ] loss:0.350 acc:0.203 
Train | Loss:0.40305 Acc:0.814
Valid | Loss:0.42480 Acc:0.802
s

## Predict and Write to csv file

In [20]:
print('loading testing data...')
test_x = load_test_date()
preprocess = Preprocess(test_x, sen_len, './w2v_all.model')
embedding = preprocess.make_embedding(load = True)
test_x = preprocess.sentence_word2idx()
test_dataset = TwitterDataset(x = test_x, y = None)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=8)

print('\nload model....')
model = torch.load('./ckpt.model')
outputs = testing(batch_size, test_loader, model, device)

#寫到csv 
tmp = pd.DataFrame({"id":[str(i) for i in range(len(test_x))], "label":outputs})
print("save csv....")
tmp.to_csv('./predict.csv',index = False)
print('Finish predicting')

loading testing data...
Get embedding...
loading word to vec model...
get words#747

/home/uscc/.pyenv/versions/3.6.8/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


get words#1475

get words#2209

get words#2946

...

get words#20044

get words#21759

get words#22649

get words#23530

get words#24694
total words : 24696
sentence count #200000
load model....
save csv....
Finish predicting
